In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F

spark = SparkSession.builder \
    .appName("PatentBranchAnalytics") \
    .config("spark.jars.packages", "org.apache.hudi:hudi-spark3.5-bundle_2.12:0.15.0") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.hudi.catalog.HoodieCatalog") \
    .config("spark.sql.hive.convertMetastoreParquet", "false") \
    .config("spark.sql.parquet.outputTimestampType", "TIMESTAMP_MICROS") \
    .config("spark.sql.datetime.java8API.enabled", "true") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .getOrCreate()

In [10]:
# Constants

filtered_patents_input_path = "../../data_source/filtered_patents"

patent_trends_table_path = "file:/Users/bhland/hive/warehouse/dashboard_analytics_results/patent_trends"  

In [11]:
patents_filtered_df = spark.read.parquet(filtered_patents_input_path)
patents_filtered_df.show(truncate=False)

patent_df_with_years = patents_filtered_df.withColumn("grant_year", year(col("patent_date"))) \
                                .withColumn("grant_month", month(col("patent_date")))


patent_df_with_years = patent_df_with_years.withColumn("grant_year", make_date(col("grant_year"), col("grant_month"), lit(1)))


+-----------+-----------+-------------------------------------------------------------------------------------------------------------------------------+----------+--------------+-----------+-----------+---------+----------------------------------+---------+
|patent_type|patent_date|patent_title                                                                                                                   |num_claims|application_id|filing_date|series_code|patent_id|branch                            |code     |
+-----------+-----------+-------------------------------------------------------------------------------------------------------------------------------+----------+--------------+-----------+-----------+---------+----------------------------------+---------+
|utility    |2018-06-19 |Filler neck closure assembly                                                                                                   |19        |15412444      |2017-01-23 |15         |10000117 |Software D

In [12]:
# Group by Year and Aggregate Counts
grant_counts = patent_df_with_years.groupBy("grant_year", "branch").agg(
    count("patent_id").alias("patent_grant_count")
)
grant_counts = grant_counts.withColumnRenamed("branch", "grant_branch")

In [13]:
final_result = grant_counts.select(
    F.col("grant_branch").alias("branch"),
    F.col("grant_year").alias("year"),
    F.coalesce(grant_counts.patent_grant_count, F.lit(0)).alias("grant_count")
)

final_result.show(truncate=False)


+----------------------------------+----------+-----------+
|branch                            |year      |grant_count|
+----------------------------------+----------+-----------+
|Data Science and Analytics        |2021-11-01|184        |
|Data Science and Analytics        |1996-04-01|22         |
|Software Development and Security |1998-07-01|16         |
|Data Science and Analytics        |1998-11-01|22         |
|Artificial Intelligence           |2006-10-01|276        |
|Artificial Intelligence           |2011-09-01|311        |
|Software Development and Security |2011-10-01|121        |
|Advanced Computing Technologies   |2011-12-01|16         |
|Artificial Intelligence           |2020-05-01|1733       |
|Artificial Intelligence           |1988-01-01|15         |
|Data Science and Analytics        |1988-03-01|4          |
|Artificial Intelligence           |1988-11-01|22         |
|Data Science and Analytics        |1993-08-01|15         |
|Networking and Distributed Systems|1997

In [14]:
df_with_year_and_month = patents_filtered_df \
    .withColumn("year", F.year("patent_date")) \
    .withColumn("month", F.month("patent_date"))  

# Group by year and branch, and calculate the average number of claims
avg_claims = df_with_year_and_month.groupBy("year","month","branch").agg(
    F.avg("num_claims").alias("avg_claims")
)

avg_claims = avg_claims.withColumn("year", make_date(col("year"), col("month"), lit(1)))

combined_df = final_result.join(
    avg_claims,
    on=["year", "branch"],
    how="left"
)

combined_df = combined_df.drop("month")

combined_df.show(truncate=False)

+----------+----------------------------------+-----------+------------------+
|year      |branch                            |grant_count|avg_claims        |
+----------+----------------------------------+-----------+------------------+
|2021-11-01|Data Science and Analytics        |184        |18.059782608695652|
|1996-04-01|Data Science and Analytics        |22         |16.636363636363637|
|1998-07-01|Software Development and Security |16         |19.5625           |
|1998-11-01|Data Science and Analytics        |22         |18.954545454545453|
|2006-10-01|Artificial Intelligence           |276        |23.44927536231884 |
|2011-09-01|Artificial Intelligence           |311        |19.591639871382636|
|2011-10-01|Software Development and Security |121        |21.90082644628099 |
|2011-12-01|Advanced Computing Technologies   |16         |17.375            |
|2020-05-01|Artificial Intelligence           |1733       |17.96537795729948 |
|1988-01-01|Artificial Intelligence           |15   

In [15]:
patent_trends_with_schema = combined_df.select(
    col("year").cast(TimestampType()).alias("timestamp"),
    col("year").cast(DateType()),
    col("branch").cast(StringType()),
    col("grant_count").cast(IntegerType()),
    col("avg_claims").cast(FloatType()),
)

patent_trends_with_schema.show(truncate=False)


+-------------------+----------+----------------------------------+-----------+----------+
|timestamp          |year      |branch                            |grant_count|avg_claims|
+-------------------+----------+----------------------------------+-----------+----------+
|2021-11-01 00:00:00|2021-11-01|Data Science and Analytics        |184        |18.059782 |
|1996-04-01 00:00:00|1996-04-01|Data Science and Analytics        |22         |16.636364 |
|1998-07-01 00:00:00|1998-07-01|Software Development and Security |16         |19.5625   |
|1998-11-01 00:00:00|1998-11-01|Data Science and Analytics        |22         |18.954546 |
|2006-10-01 00:00:00|2006-10-01|Artificial Intelligence           |276        |23.449276 |
|2011-09-01 00:00:00|2011-09-01|Artificial Intelligence           |311        |19.59164  |
|2011-10-01 00:00:00|2011-10-01|Software Development and Security |121        |21.900826 |
|2011-12-01 00:00:00|2011-12-01|Advanced Computing Technologies   |16         |17.375    |

In [16]:
patent_trends_hudi_options = {
    'hoodie.table.name': 'patent_trends',
    'hoodie.datasource.write.recordkey.field': 'year,branch',
    'hoodie.datasource.write.precombine.field': "grant_count",
    'hoodie.datasource.write.table.name': 'patent_trends',
    'hoodie.datasource.write.table.type': 'COPY_ON_WRITE',
    'hoodie.datasource.write.operation': 'insert',
    'hoodie.upsert.shuffle.parallelism': 2,
    'hoodie.insert.shuffle.parallelism': 2,
}
patent_trends_with_schema.write.format("org.apache.hudi").options(**patent_trends_hudi_options).mode("overwrite").save(patent_trends_table_path)

print("The Patents Trends results have been successfully written to the hudi table in hive warehouse.")

24/12/01 18:01:37 WARN HoodieSparkSqlWriterInternal: hoodie table at file:/Users/bhland/hive/warehouse/dashboard_analytics_results/patent_trends already exists. Deleting existing data & overwriting with new data.


The Patents Trends results have been successfully written to the hudi table in hive warehouse.


24/12/01 18:01:39 WARN HoodieSparkSqlWriterInternal: Closing write client
